In [19]:
import requests
import pandas as pd

queries = {
    "series_with_start_date": """
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX wd: <http://www.wikidata.org/entity/>
        SELECT ?series ?seriesLabel ?startDate WHERE {
          ?series wdt:P31 wd:Q5398426.
          OPTIONAL { ?series wdt:P577 ?startDate. }
          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
          FILTER (BOUND(?startDate))
        } ORDER BY ?startDate
        LIMIT 10
    """,
    "genre": """
        SELECT ?series ?seriesLabel ?genreLabel WHERE {
          ?series wdt:P31 wd:Q5398426.
          ?series wdt:P136 ?genre.
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        } LIMIT 10
    """,
    "actors": """
        SELECT ?series ?seriesLabel ?actorLabel WHERE {
          ?series wdt:P31 wd:Q5398426.
          ?series wdt:P161 ?actor.
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        } LIMIT 10
    """,
    "directors": """
        SELECT ?series ?seriesLabel ?directorLabel WHERE {
          ?series wdt:P31 wd:Q5398426.
          ?series wdt:P57 ?director.
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        } LIMIT 10
    """,
    "country_of_origin": """
        SELECT ?series ?seriesLabel ?countryLabel WHERE {
          ?series wdt:P31 wd:Q5398426.
          ?series wdt:P495 ?country.
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        } LIMIT 10
    """,
    "original_language": """
        SELECT ?series ?seriesLabel ?languageLabel WHERE {
          ?series wdt:P31 wd:Q5398426.
          ?series wdt:P364 ?language.
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        } LIMIT 10
    """,
    "number_of_seasons_episodes": """
        SELECT ?series ?seriesLabel ?numberOfSeasons ?numberOfEpisodes WHERE {
          ?series wdt:P31 wd:Q5398426.
          ?series wdt:P2437 ?numberOfSeasons.
          ?series wdt:P1113 ?numberOfEpisodes.
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        } LIMIT 10
    """,
    "broadcasters": """
        SELECT ?series ?seriesLabel ?broadcasterLabel WHERE {
          ?series wdt:P31 wd:Q5398426.
          ?series wdt:P449 ?broadcaster.
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        } LIMIT 10
    """,
    "imdb_rotten_tomatoes_ids": """
        SELECT ?series ?seriesLabel ?imdbId ?rottenTomatoesId WHERE {
          ?series wdt:P31 wd:Q5398426.
          OPTIONAL { ?series wdt:P345 ?imdbId. }
          OPTIONAL { ?series wdt:P1258 ?rottenTomatoesId. }
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        } LIMIT 10
    """,
    "oscar": """
        SELECT ?film ?filmLabel ?award ?awardLabel WHERE {
          ?film wdt:P31 wd:Q11424; # Type: Film
                wdt:P166 ?award. # A reçu une récompense
          ?award wdt:P31 wd:Q19020. # Type de récompense: Oscar
          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
        } LIMIT 100
    """
        }

def query_wikidata(query_name):
    if query_name not in queries:
        return pd.DataFrame()  # Retourne un DataFrame vide si la requête n'existe pas
    
    sparql_query = queries[query_name]
    url = "https://query.wikidata.org/sparql"
    
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, params={'format': 'json', 'query': sparql_query})
    
    if response.status_code != 200:
        print("Erreur lors de la requête SPARQL", response.status_code)
        return pd.DataFrame()  # Retourne un DataFrame vide en cas d'erreur
    
    data = response.json()
    results = data['results']['bindings']
    rows = []
    
    for result in results:
        row = {key: result[key]['value'] for key in result.keys()}
        rows.append(row)
    
    df = pd.DataFrame(rows)
    
    return df

In [6]:
query_wikidata("original_language").head()

,series,seriesLabel,languageLabel
0,http://www.wikidata.org/entity/Q33323,Everybody Hates Chris,English
1,http://www.wikidata.org/entity/Q34316,Doctor Who,British English
2,http://www.wikidata.org/entity/Q34659,My Name Is Earl,English
3,http://www.wikidata.org/entity/Q34669,Raising Hope,English
4,http://www.wikidata.org/entity/Q35023,My Babysitter's a Vampire,English


In [7]:
query_wikidata("genre").head()

,series,seriesLabel,genreLabel
0,http://www.wikidata.org/entity/Q494,Beakman's World,comedy television series
1,http://www.wikidata.org/entity/Q723,Rookie Blue,drama
2,http://www.wikidata.org/entity/Q723,Rookie Blue,police procedural
3,http://www.wikidata.org/entity/Q723,Rookie Blue,LGBTI+ related TV series
4,http://www.wikidata.org/entity/Q961,More Than Life at Stake,espionage television series


In [8]:
query_wikidata("actors").head()

,series,seriesLabel,actorLabel
0,http://www.wikidata.org/entity/Q494,Beakman's World,Eliza Schneider
1,http://www.wikidata.org/entity/Q494,Beakman's World,Senta Moses
2,http://www.wikidata.org/entity/Q494,Beakman's World,Alanna Ubach
3,http://www.wikidata.org/entity/Q494,Beakman's World,Paul Zaloom
4,http://www.wikidata.org/entity/Q494,Beakman's World,Mark Ritts


In [9]:
query_wikidata("directors").head()

,series,seriesLabel,directorLabel
0,http://www.wikidata.org/entity/Q961,More Than Life at Stake,Janusz Morgenstern
1,http://www.wikidata.org/entity/Q961,More Than Life at Stake,Andrzej Konic
2,http://www.wikidata.org/entity/Q990,Czterej pancerni i pies,Konrad Nałęcki
3,http://www.wikidata.org/entity/Q1079,Breaking Bad,Vince Gilligan
4,http://www.wikidata.org/entity/Q1079,Breaking Bad,Bryan Cranston


In [10]:
query_wikidata("country_of_origin").head()

,series,seriesLabel,countryLabel
0,http://www.wikidata.org/entity/Q173836,Ein starkes Team,Germany
1,http://www.wikidata.org/entity/Q174390,State of Play,United Kingdom
2,http://www.wikidata.org/entity/Q174972,Touched by an Angel,United States of America
3,http://www.wikidata.org/entity/Q175553,Time Warp,United States of America
4,http://www.wikidata.org/entity/Q176027,Tales of the Unexpected,United Kingdom


In [11]:
query_wikidata("original_language").head()

,series,seriesLabel,languageLabel
0,http://www.wikidata.org/entity/Q216919,Six Feet Under,English
1,http://www.wikidata.org/entity/Q217246,The Suite Life of Zack & Cody,English
2,http://www.wikidata.org/entity/Q217276,Jonas,English
3,http://www.wikidata.org/entity/Q217567,Rosa Roth,German
4,http://www.wikidata.org/entity/Q217902,Animal Practice,English


In [12]:
query_wikidata("number_of_seasons_episodes").head()

,series,numberOfSeasons,numberOfEpisodes,seriesLabel
0,http://www.wikidata.org/entity/Q202727,1,13,Meego
1,http://www.wikidata.org/entity/Q210468,1,21,Vegas
2,http://www.wikidata.org/entity/Q210505,1,18,Chase
3,http://www.wikidata.org/entity/Q210665,1,18,Malibu Country
4,http://www.wikidata.org/entity/Q210683,1,13,Partners


In [13]:
query_wikidata("broadcasters").head()

,series,seriesLabel,broadcasterLabel
0,http://www.wikidata.org/entity/Q115874,"El Zorro, la espada y la rosa",Telemundo
1,http://www.wikidata.org/entity/Q115945,Between Love and Hatred,Las Estrellas
2,http://www.wikidata.org/entity/Q117396,CSI: Crime Scene Investigation,CBS
3,http://www.wikidata.org/entity/Q117590,"Beverly Hills, 90210",Fox Broadcasting Company
4,http://www.wikidata.org/entity/Q118689,Birds of Prey,The WB


In [14]:
query_wikidata("imdb_rotten_tomatoes_ids").head()

,series,imdbId,rottenTomatoesId,seriesLabel
0,http://www.wikidata.org/entity/Q216919,tt0248654,tv/six-feet-under,Six Feet Under
1,http://www.wikidata.org/entity/Q217246,tt0426371,NaN,The Suite Life of Zack & Cody
2,http://www.wikidata.org/entity/Q217276,tt1044418,NaN,Jonas
3,http://www.wikidata.org/entity/Q217567,tt0806904,NaN,Rosa Roth
4,http://www.wikidata.org/entity/Q217902,tt2246569,tv/animal-practice,Animal Practice


In [21]:
query_wikidata("oscar").head()

,award,film,filmLabel,awardLabel
0,http://www.wikidata.org/entity/Q102427,http://www.wikidata.org/entity/Q59653,Argo,Academy Award for Best Picture
1,http://www.wikidata.org/entity/Q102427,http://www.wikidata.org/entity/Q60072,The Sound of Music,Academy Award for Best Picture
2,http://www.wikidata.org/entity/Q102427,http://www.wikidata.org/entity/Q61696,Midnight Cowboy,Academy Award for Best Picture
3,http://www.wikidata.org/entity/Q102427,http://www.wikidata.org/entity/Q62665,The Sting,Academy Award for Best Picture
4,http://www.wikidata.org/entity/Q102427,http://www.wikidata.org/entity/Q63026,The English Patient,Academy Award for Best Picture
